In [5]:
from features_EOC import features_EOC
from features_EOS import features_EOS
from features_Pred import features_Pred
from features_slope import features_slope
from features_DFA import features_DFA
from features_AVC import features_AVC
from features_avc_std_dist import features_avc_std_dist
import mne
import numpy as np
import glob
from saving_utils import path_to_names


###########
### import data
###########

paths = glob.glob('/Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/hand_cleaning/ALL/10s/*.fif')
paths.sort()
path = paths[40]

mne_epochs_raw = mne.read_epochs(path)
mne_epochs_32 = mne_epochs_raw.copy()

mne_epochs_raw.pick_types(eeg=True)

bad_chans = list(mne_epochs_32.info['bads'])
mne_epochs_32.interpolate_bads(reset_bads=True)
mne_epochs_32.pick_types(eeg=True)
all_chans = mne_epochs_32.ch_names
good_chans = [ch for ch in all_chans if ch not in bad_chans]

channel_indices = {name: idx for idx, name in enumerate(mne_epochs_32.ch_names)}
bad_indices = [channel_indices[ch] for ch in bad_chans]
good_indices = [channel_indices[ch] for ch in good_chans]


path_raw = path.replace('_epo.fif', '_raw.fif').replace('10s', 'raw')

raw_raw = mne.io.read_raw_fif(path_raw, preload=True)
bad_chans_raw = list(raw_raw.info['bads'])
assert bad_chans_raw == bad_chans, 'bad channels do not match'
raw_32 = raw_raw.copy()
raw_32.interpolate_bads(reset_bads=True)
raw_raw.pick_types(eeg=True)
raw_32.pick_types(eeg=True)



############
### Analysis
############

fbands = [[1,45], [1, 4], [4, 8], [8, 13], [13, 30], [30, 45]]

# on EPOCH
K_median, K_median_interpolated, Freq, Nopeak, eoc_results, eoc_results_interpolated = features_EOC(mne_epochs_32, k_type='flex', hfrequ=None, max_trials=5, bad_indices=bad_indices, good_indices=good_indices)

PCF_mean, OR_mean, eos_results = features_EOS (mne_epochs_32, minfreq=1, maxfreq=10, fs=256, max_trials=5)
for fband in fbands:
    PCF_mean_interpolated, OR_mean_interpolated, eos_results_interpoalted = features_EOS (mne_epochs_32, minfreq=fband[0], maxfreq=fband[1], fs=256, max_trials=5)

Lyaps_max, Dims_mean, Ent_mean, LZC_mean, KDF_mean, pred_results, Lyaps_max_int, Dims_mean_int, Ent_mean_int, LZC_mean_int, KDF_mean_int, pred_results_int  = features_Pred (mne_epochs_32, lfreq=0.5, hfreq=40, fs=256, max_trials=3, bad_indices=bad_indices)

slope_id, slope_id_interpoalted, Slope_space_id, Slope_space_id_interpoalted, psds_mean, psds_mean_interpolated, psds, psds_interpoalted, freqs = features_slope (mne_epochs_32, lfreq=0.5, hfreq=40, fs=256, max_trials=3, bad_indices=bad_indices)


# on RAW
for fband in fbands:
    HURST_FH, HURST_DFA, results, HURST_FH_interpoalted, HURST_DFA_interpoalted, results_interpolated = features_DFA(raw_32, lfreq=fband[0], hfreq=fband[1], fs=256, max_s=50, bad_indices=bad_indices)
out, avls = features_AVC (raw_32, bin_threshold=0.0005, max_iei=0.2, fs=256, max_s=200, lfreq=0.5, hfreq=40)
part_hist_x, part_hist_y, part_hist_x_raw, part_hist_y_raw =  features_avc_std_dist (raw_32, max_s=50, fs=256, lfreq=0.5, hfreq=40)


Reading /Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/hand_cleaning/ALL/10s/0_01_NA_rposo_03_003_0106_008_0080_034_0102_epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available
Not setting metadata
8 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 29 sensor positions
Interpolating 3 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Opening raw data file /Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/hand_cleaning/ALL/raw/0_01_NA_rposo_03_003_0106_008_0080_034_0102_raw.fif...
    Range : 512 ... 27738 =      2.000 ...   108.352 secs
Ready.
Reading 0 ... 27226  =      0.000 ...   

/Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/python_master_function/features_EOC.py:102: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = mne_epochs.get_data()
/Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/python_master_function/features_Pred.py:65: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = epochs_filt.get_data()
/Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/python_master_function/features_slope.py:24: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = mne_epochs.get_data()
Assuming nested distributions


number of elements: 142


/opt/anaconda3/lib/python3.8/site-packages/powerlaw.py:1615: RuntimeWarning: invalid value encountered in divide
  CDF = CDF/norm
'nan' in fit cumulative distribution values.
Likely underflow or overflow error: the optimal fit for this distribution gives values that are so extreme that we lack the numerical precision to calculate them.
Assuming nested distributions
/opt/anaconda3/lib/python3.8/site-packages/edgeofpy/avalanche.py:702: RuntimeWarning: divide by zero encountered in divide
  br = np.ma.masked_invalid(events[1:] / events[:-1]).mean()
/opt/anaconda3/lib/python3.8/site-packages/edgeofpy/avalanche.py:702: RuntimeWarning: invalid value encountered in divide
  br = np.ma.masked_invalid(events[1:] / events[:-1]).mean()


In [16]:
Lyaps_max, Dims_mean, Ent_mean, LZC_mean, KDF_mean

(0.12846904258721797,
 3.0114942528735633,
 0.12356790834962803,
 0.03716024352264125,
 1.324681799476467)

In [27]:
out

{'mean_iei': [0.31631324404761907],
 'tau': [1.6790456853816145],
 'tau_dist': ['T'],
 'tau_dist_TR': [8.434363225513678],
 'alpha': [1.6640447695563505],
 'alpha_dist': ['T'],
 'alpha_dist_TR': [9.161288264709778],
 'third': [1.4830384080077854],
 'dcc_cn': [0.5051295811765445],
 'avl_br': [0.04047912137325089],
 'br': [0.6527861027861028],
 'rep_dissimilarity_avg': [0.2263718820861678],
 'rep_size': [0.98725529804973],
 'fano': [3.051035393190822],
 'chi_test': [0.001722909948591798],
 'chi_notest': [0.0017229099485917981],
 'sig_length': [106.35546875],
 'len_avls': [142],
 'data_mean': [1.4033033072315858e-05],
 'data_std': [2.6361482415751573e-05]}

In [20]:
Dims_mean_int

3.0114942528735633

In [21]:
Ent_mean_int

0.12356790834962803

In [22]:
LZC_mean_int

0.03716024352264125

In [26]:
Slope_space_id

array([-0.13440123, -0.13185031, -0.10740272, -0.11470919, -0.10918178,
       -0.19775803, -0.04895576,         nan,  0.04608382, -0.11427639,
       -0.26265943,  0.00781533,  0.04353978,  0.09543341, -0.41761753,
       -0.09540119, -0.05219371,  0.03755617, -0.07763069, -0.28712662,
       -0.09301215, -0.02044054, -0.04561614,         nan, -0.12723368,
               nan, -0.0280922 , -0.02124284, -0.05402839, -0.15597118,
       -0.09822967, -0.07139648])